In [1]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from modules.data.data_loaders import DataLoaderManager
from modules.preprocessing.smiles import SMILESChecker
from modules.preprocessing.descriptors import *

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\Willy\anaconda3\envs\ml\python.exe"
  * The NumPy version is: "1.18.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: No se puede encontrar el módulo especificado.


In [ ]:
RANDOM_SEED = 42

DATA_PATH = 'data/Tox21/tox21_10k_data_all.sdf'

In [ ]:
# 0. Load data (class DataLoaderManager)
data_loader = DataLoaderManager()
data = data_loader.load(
    path=DATA_PATH,
    removeHs=False
)

In [ ]:
# 0.1. Custom preprocessing (specific for each source)
target_var = 'SR-p53'
data = data[~data[target_var].isnull()]

In [ ]:
# TODO: for testing purposes only
data = data[:1000]

In [ ]:
# 1. Preprocess data
# 1.1. Check and sanitize data
smiles_pipe = Pipeline(steps=[
    ('SMILESChecker', SMILESChecker())
])
data['SMILES'] = smiles_pipe.fit_transform(
    X=data['SMILES'].to_numpy()
)

# Recalculate mol from curated SMILES
data['Molecule'] = [Chem.MolFromSmiles(smiles)
                    for smiles in data['SMILES']]

In [ ]:
# 1.2. Descriptors calculation
desc_pipe = DescriptorPipeline(mol_column='Molecule', steps=[
    ('Mordred', DescriptorMordred())
])
data = desc_pipe.fit_transform(X=data)

In [ ]:
# 1.3. Hold-out split
train_data, test_data = train_test_split(
    data.copy(),
    test_size=0.33,
    random_state=RANDOM_SEED
)

In [ ]:
# 1.4 EDA (mass, charge, LogP, etc.)
variables = {
    'x': 'MW',
    'y': 'nHetero',
    'z': 'SLogP',
    'color': target_var
}

import plotly.express as px

fig = px.scatter_3d(
    data_frame=data,
    x=variables['x'],
    y=variables['y'],
    z=variables['z'],
    color=variables['color'],
    template='plotly_white',
    height=750,
    width=900,
    title='Initial EDA'
)
fig.show()

In [ ]:
# 1.5. Data enrichment
# NOTE: perform enrichment after hold-out to avoid data leakage from train
# to test, as it can happen with some techniques such as Active Learning
# 1.5.1. Enrichment from external databases
# 1.5.2. Post-enrichment EDA (mass, charge, LogP, etc.)

# 2. Features
# 2.1. Feature engineering + EDA (new variables)
pass

# 2.2. Feature selection + EDA (selected variables)
pass

# 3. Label enrichment
# 3.1. Generative models
# 3.2. Active Learning (validation of generative models and data enrichment)

# 4. Fit model (class ModelSelector)
# 4.1. Hyperparameters selection
# 4.2. Validation strategy (CV, NestedCV, etc.)

# 5. Evaluate model (class ModelEvaluator)
# 5.1. Metrics
# 5.2. Visualization of predictions, generated data, etc.
# 5.3. Explainability:
#   - Shapley
#   - Applicability domain